In [1]:
from matplotlib import transforms
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import cv2 as cv
import pickle
import gzip
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

import torchvision.transforms as T
import torch

In [2]:
def train_validate_split(X_train,y_train):
    X_tr, y_tr = shuffle(X_train,y_train)
    _, X_val, _, y_val = train_test_split(X_tr,y_tr,test_size=0.1)
    return X_tr,y_tr,X_val,y_val

def one_hot_encoding(train_labels,test_labels,num_classes):
    lb = LabelEncoder()
    train_labels = lb.fit_transform(train_labels)
    test_labels = lb.fit_transform(test_labels)
    train_label_ohc = []
    test_label_ohc = []
    for l in train_labels:
            train_label_ohc.append(np.eye(num_classes)[l])
    for l in test_labels:
            test_label_ohc.append(np.eye(num_classes)[l])
    return train_label_ohc,test_label_ohc

In [3]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


class CIFAR10():

    def __init__(self):
        self.path_to_dir = '../content/drive/MyDrive/datasets/cifar-10-batches-py'
        self.meta_data = unpickle(self.path_to_dir + "/batches.meta")
        images, labels, test_images, test_labels = self.__load()
        self.__prepare_data(images,labels,test_images,test_labels)

    def __load(self):
        #num_cases_per_batch: 10000
        #label_names': airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck
        #num_vis: 3072
        label_names = self.meta_data[b'label_names']
        label_names = np.array(label_names)
        data = []
        labels = []
        self.labels_ohc = []
        for i in range(1,6):
            #Data,filenames und label extrahieren
            curr_batch = unpickle(self.path_to_dir + "/data_batch_{}".format(i))
            #keys: b'batch_label', b'labels', b'data', b'filenames'
        
            data.append(curr_batch[b'data']) #shape: 10000,3072 -> 3,32,32
            
            labels += curr_batch[b'labels']
        
        cifar_test_batch = unpickle(self.path_to_dir + "/test_batch")
        data.append(cifar_test_batch[b'data'])
        labels += cifar_test_batch[b'labels']
                
        #Formatierung der Liste in 3,32,32 das nn mit den Daten arbeiten kann
        data = np.reshape(data,(len(data[0])*6,3072))
        data = data.reshape(len(data),3,32,32)
        data = np.rollaxis(data, 1, 4)
        train_data = data[0:50000]
        train_labels = labels[0:50000]
        test_data = data[50000:]
        test_labels = labels[50000:]
        return train_data,train_labels,test_data,test_labels
    
    def __prepare_data(self,train_data,train_labels,X_test,test_labels):
        train_lb,y_test = one_hot_encoding(train_labels,test_labels,10)
        X_train,y_train,X_val,y_val = train_validate_split(train_data,train_lb)
        
        self.dict_images = {'X_train':X_train,
                            'X_test': X_test,
                            'X_val': X_val,
                        }
        self.dict_labels = {'y_train': y_train,
                            'y_test': y_test,
                            'y_val': y_val
                        }

    def get_data(self,what_is_needed):
        if what_is_needed == 'train':
            return (self.dict_images['X_train'],self.dict_labels['y_train'])
        elif what_is_needed == 'test':
            return (self.dict_images['X_test'],self.dict_labels['y_test'])
        elif what_is_needed == 'val':
            return (self.dict_images['X_val'],self.dict_labels['y_val'])
        elif what_is_needed == 'all':
            return (self.dict_images,self.dict_labels)
          
    def print_data(self):

        plot, ax = plt.subplots(3,3)
        for i in range(3):
            for j in range(3):
                idx = np.random.randint(0,self.X_train.shape[0])

                ax[i,j].imshow(self.X_train[idx])
                ax[i,j].set_xlabel(self.y_train[idx])
                ax[i,j].get_yaxis().set_visible(False)
        plt.show()

    def get_mean_std(self):
        mean = [0.49139968, 0.48215841, 0.44653091]
        std =  [0.2469767,  0.24336646, 0.26144247]

In [4]:
from torch.nn.functional import normalize
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

class CIFAR10():

    def __init__(self):
        self.LABELS = {b'airplane':0, b'automobile':1, b'bird':2, b'cat':3, b'deer':4, b'dog':5, b'frog':6, b'horse':7,
 b'ship':8, b'truck':9}
        self.path_to_dir = '../files/cifar-10-batches-py'
        self.meta_data = unpickle(self.path_to_dir + "/batches.meta")

        self.X_train,self.y_train,self.X_test,self.y_test = \
            self.__load()


    def __load(self):
        #num_cases_per_batch: 10000
        #label_names': airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck
        #num_vis: 3072
        label_names = self.meta_data[b'label_names']
        label_names = np.array(label_names)
        data = []
        labels = []
        self.labels_ohc = []
        for i in range(1,6):
            #Data,filenames und label extrahieren
            curr_batch = unpickle(self.path_to_dir + "/data_batch_{}".format(i))
            #keys: b'batch_label', b'labels', b'data', b'filenames'
        
            data.append(curr_batch[b'data']) #shape: 10000,3072 -> 3,32,32
            
            labels += curr_batch[b'labels']
        
        cifar_test_batch = unpickle(self.path_to_dir + "/test_batch")
        data.append(cifar_test_batch[b'data'])
        labels += cifar_test_batch[b'labels']

        for l in labels:
            self.labels_ohc.append(np.eye(10)[l])
        
        #Formatierung der Liste in 3,32,32 das nn mit den Daten arbeiten kann
        data = np.reshape(data,(len(data[0])*6,3072))
        data = data.reshape(len(data),3,32,32)
        data = np.rollaxis(data, 1, 4)
        train_data = data[0:50000]
        train_labels = self.labels_ohc[0:50000]
        test_data = data[50000:]
        test_labels = self.labels_ohc[50000:]

        return train_data,train_labels,test_data,test_labels
    
    def get_data(self):
        return self.X_train,self.y_train,self.X_test,self.y_test
           
    def print_data(self):

        plot, ax = plt.subplots(3,3)
        for i in range(3):
            for j in range(3):
                idx = np.random.randint(0,self.X_train.shape[0])

                ax[i,j].imshow(self.X_train[idx])
                ax[i,j].set_xlabel(self.y_train[idx])
                ax[i,j].get_yaxis().set_visible(False)
        plt.show()

    def get_mean_std(self):
        mean = [0.49139968, 0.48215841, 0.44653091]
        std =  [0.2469767,  0.24336646, 0.26144247]


In [5]:
cif = CIFAR10()

FileNotFoundError: [Errno 2] No such file or directory: '../files/cifar-10-batches-py/batches.meta'

Initialize Custom Dataset and assign X_train and X_test data

In [ ]:
from torchvision.transforms.transforms import ToPILImage
import numpy as np
from sympy import Ci
import torch
import torchvision.transforms as T
from torch.utils.data import Dataset

class Cifar10Dataset():
    def __init__(self,images,labels):
        self.images = images
        self.labels = labels
        self.transform = T.Compose([
            T.ToTensor()
        ])
    
    def __len__(self):
        return len(self.images)

    def __getitem__(self,idx):
        if self.transform:
            images = self.transform(self.images[idx])
            normalize = T.Normalize( [0.49139968, 0.48215841, 0.44653091],[0.2469767,  0.24336646, 0.26144247])
            norm = normalize(images)
            images = norm
        return (images,self.labels[idx])

load split data into dataloader

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

#epoche 15: 0.1 lr 0.1
'''class CNN_CIFAR(nn.Module):
    def __init__(self):
        super(CNN_CIFAR,self).__init__()
        #conv layers
        self.conv1 = nn.Conv2d(3,32,3,1)

        #fc layers
        self.fc1 = nn.Linear(28800,10)
    
    def forward(self,x):
      x = F.relu(self.conv1(x))
      x = x.reshape(x.shape[0],-1)
      x = self.fc1(x)
      return x'''
# Train acc: 0.93006 Train loss: 0.21952689442121304 lr 0.001
'''class CNN_CIFAR(nn.Module):
    def __init__(self):
        super(CNN_CIFAR,self).__init__()
        #conv layers
        self.conv1 = nn.Conv2d(3,128,3,1)

        #fc layers
        self.fc1 = nn.Linear(115200,10)
    
    def forward(self,x):
      x = F.relu(self.conv1(x))
      x = x.reshape(x.shape[0],-1)
      x = self.fc1(x)
      return x'''
# acc: 0.91088 Train loss: 0.2775786456513608 lr 0.001
'''class CNN_CIFAR(nn.Module):
    def __init__(self):
        super(CNN_CIFAR,self).__init__()
        #conv layers
        self.conv1 = nn.Conv2d(3,64,3,1)

        #fc layers
        self.fc1 = nn.Linear(57600,10)
    
    def forward(self,x):
      x = F.relu(self.conv1(x))
      x = x.reshape(x.shape[0],-1)
      x = self.fc1(x)
      return x'''
#Train acc: 0.939 Train loss: 0.18734320437258764 lr 0.001
'''class CNN_CIFAR(nn.Module):
    def __init__(self):
        super(CNN_CIFAR,self).__init__()
        #conv layers
        self.conv1 = nn.Conv2d(3,128,5,1)

        #fc layers
        self.fc1 = nn.Linear(25088,10)
    
    def forward(self,x):
      x = F.relu(self.conv1(x))
      x = x.reshape(x.shape[0],-1)
      x = self.fc1(x)
      return x
'''
#Train acc: 0.82806 Train loss: 0.5260674791708588 lr 0.0003
'''class CNN_CIFAR(nn.Module):
    def __init__(self):
        super(CNN_CIFAR,self).__init__()
        #conv layers
        self.conv1 = nn.Conv2d(3,128,5,1)
        self.conv2 = nn.Conv2d(128,32,3,1)

        #fc layers
        self.fc1 = nn.Linear(21632,10)
    
    def forward(self,x):
      x = F.relu(self.conv1(x))
      x = F.relu(self.conv2(x))
      x = x.reshape(x.shape[0],-1)
      x = self.fc1(x)
      return x'''
#Train acc: 0.97834 Train loss: 0.07318448818677437 lr 0.0003
'''class CNN_CIFAR(nn.Module):
    def __init__(self):
        super(CNN_CIFAR,self).__init__()
        #conv layers
        self.conv1 = nn.Conv2d(3,128,5,1)
        self.conv2 = nn.Conv2d(128,64,3,1)

        #fc layers
        self.fc1 = nn.Linear(43264,10)
    
    def forward(self,x):
      x = F.relu(self.conv1(x))
      x = F.relu(self.conv2(x))
      x = x.reshape(x.shape[0],-1)
      x = self.fc1(x)
      return x'''
#Train acc: 0.977 Train loss: 0.06664528534313287 lr 0.001
'''class CNN_CIFAR(nn.Module):
    def __init__(self):
        super(CNN_CIFAR,self).__init__()
        #conv layers
        self.conv1 = nn.Conv2d(3,64,3,1)
        self.conv2 = nn.Conv2d(64,32,3,1)
        #fc layers
        self.fc1 = nn.Linear(25088,10)
    
    def forward(self,x):
      x = F.relu(self.conv1(x))
      x = F.relu(self.conv2(x))
      x = x.reshape(x.shape[0],-1)
      x = self.fc1(x)
      return x'''
#Train acc: 0.89956 Train loss: 0.33078178741363196
'''class CNN_CIFAR(nn.Module):
    def __init__(self):
        super(CNN_CIFAR,self).__init__()
        #conv layers
        self.conv1 = nn.Conv2d(3,16,3,1)

        #fc layers
        self.fc1 = nn.Linear(14400,10)
    
    def forward(self,x):
      x = F.relu(self.conv1(x))
      x = x.reshape(x.shape[0],-1)
      x = self.fc1(x)
      return x'''
#Train acc: 0.87052 Train loss: 0.39550972418221986
class CNN_CIFAR(nn.Module):
    def __init__(self):
        super(CNN_CIFAR,self).__init__()
        #conv layers
        self.conv1 = nn.Conv2d(3,32,3,1)

        #fc layers
        self.fc1 = nn.Linear(28800,10)
    
    def forward(self,x):
      x = F.relu(self.conv1(x))
      x = x.reshape(x.shape[0],-1)
      x = self.fc1(x)
      return x

class CNN_CIFAR(nn.Module):
    def __init__(self):
        super(CNN_CIFAR,self).__init__()
        #conv layers
        self.conv1 = nn.Conv2d(3,64,3,1)
        self.conv2 = nn.Conv2d(64,32,5,1)
        #fc layers
        self.fc1 = nn.Linear(21632,10)
    
    def forward(self,x):
      x = F.relu(self.conv1(x))
      x = F.relu(self.conv2(x))
      x = x.reshape(x.shape[0],-1)
      x = self.fc1(x)
      return x

In [ ]:
from torchsummary import summary
model = CNN_CIFAR()
model.to('cuda')
summary(model, (3, 32, 32))

In [ ]:
import numpy as np
from sklearn.utils import shuffle
import torch.nn as nn
import torch.nn.functional as F


import torch

import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
x = current_time.replace(':','_')


#device setup 
device =  'cuda:0' if torch.cuda.is_available() else 'cpu'
if device == 'cuda:0':
    print('running on gpu')

writer = SummaryWriter()
#Params 0.1 0.03 0.001 0.0003
lr = 0.001
epochs = 200
batch_size = 128

model = CNN_CIFAR()
optimizer = optim.Adam(model.parameters(),lr=lr)
criterion = nn.CrossEntropyLoss()
model.to(device)
writer = SummaryWriter(f'/runs/cifar_conv64_32_1fc_lr0.001_kernel3_5s1')
#Data
dat = CIFAR10()

#dataset
ds = Cifar10Dataset


#get data for dl
X_train,y_train = dat.get_data('train')
X_val,y_val = dat.get_data('val')
train = ds(X_train,y_train)
test = ds(X_val,y_val)

trains = torch.utils.data.DataLoader(dataset=train,batch_size=batch_size,shuffle=False)
tests = torch.utils.data.DataLoader(dataset=test,batch_size=batch_size,shuffle=False)

def evaluate(X, y, train=False):
        if train:
            model.zero_grad()

        scores = model(X)
        matches = [torch.argmax(i) == torch.argmax(j) for i,j in zip(scores,y)]
        acc = matches.count(True)/len(matches)

        loss = criterion(scores,y)
        if train:
            loss.backward()
            optimizer.step()
        return acc,loss.item()

def train(epoch,trainss):
        model.train()
        acc = 0.0
        loss = 0.0

        for idx,(data,targets) in enumerate(trainss):
            
            data = data.to(device=device)
            targets = targets.to(device=device)
            
            accs, losses = evaluate(data,targets,train=True)
            loss += losses * data.size(0)
            acc += accs * data.size(0)
        
        mean_acc = acc / len(trainss.dataset)
        mean_loss = loss / len(trainss.dataset)
        writer.add_scalar('Mean Accuracy Train',mean_acc,epoch)
        writer.add_scalar('Mean Loss Train',mean_loss,epoch)
        return mean_acc, mean_loss
def test(epoch,testss):
        acc = 0.0
        loss = 0.0
        model.eval()
        with torch.no_grad():
            for (data,targets) in testss:
                data = data.to(device=device)
                targets = targets.to(device=device)

                accs, losses = evaluate(data,targets,train=False)
                loss += losses * data.size(0)
                acc += accs * data.size(0)
        mean_acc = acc / len(testss.dataset)
        mean_loss = loss / len(testss.dataset)
        writer.add_scalar('Mean Accuracy Test',mean_acc,epoch)
        writer.add_scalar('Mean Loss Test',mean_loss,epoch)
        return mean_acc, mean_loss
for epoch in range(epochs):
    print(f'Epoch: [{epoch+1} / {epochs}]')
    train_acc,train_loss = train(epoch,trains)

    test_acc,test_loss = test(epoch,tests)
    print(f'Test acc: {test_acc} Test loss: {test_loss} Train acc: {train_acc} Train loss: {train_loss}')

0
tensor([ 0.2279, -0.2810, -0.0056])
tensor([0.8466, 1.3092, 1.3283])
tensor([0.7168, 1.7141, 1.7643])
tensor([-0.6809, -0.0990,  0.6998])
tensor([0.3697, 0.3027, 0.2945])
tensor([0.1367, 0.0917, 0.0867])
tensor([-0.9863, -0.9208, -1.1283])
tensor([0.6548, 0.6276, 0.5527])
tensor([0.4287, 0.3939, 0.3054])
tensor([0.0808, 0.2837, 0.4689])
tensor([0.8243, 0.7789, 0.7609])
tensor([0.6795, 0.6067, 0.5789])
tensor([0.8842, 0.4340, 0.5502])
tensor([1.0051, 1.3289, 1.2280])
tensor([1.0103, 1.7660, 1.5081])
tensor([-0.4335, -0.7780, -0.7654])
tensor([0.3868, 0.2660, 0.2496])
tensor([0.1496, 0.0708, 0.0623])
tensor([-0.4240, -0.4888, -0.7233])
tensor([1.2266, 1.1415, 0.9637])
tensor([1.5045, 1.3029, 0.9288])
tensor([0.9653, 1.0176, 1.0835])
tensor([0.7412, 0.7522, 0.7002])
tensor([0.5493, 0.5658, 0.4902])
tensor([ 0.2901, -0.1079, -0.1016])
tensor([1.1283, 1.2103, 0.9229])
tensor([1.2732, 1.4648, 0.8517])
tensor([-0.6896, -0.6413, -0.1671])
tensor([0.4471, 0.5367, 0.4135])
tensor([0.1999, 0.28

UnboundLocalError: local variable 'global_step_train' referenced before assignment